In [21]:
from wordcloud import WordCloud
import pandas as pd

data = pd.read_csv('dataset/train/train.csv')


###
###
###
# Function to extract words from the 'word' key in entity dictionaries
def extract_words(entity_column):
    words = [entity['word'] for entity in entity_column if entity is not None]
    return ' '.join(words)

# Extract words from 'subject_entity' and 'object_entity' columns
subject_words = extract_words(data['subject_entity'])
object_words = extract_words(data['object_entity'])

# Generate word clouds
wordcloud_subject = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                min_font_size = 10).generate(subject_words)

wordcloud_object = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                min_font_size = 10).generate(object_words)

# Plot the word clouds
plt.figure(figsize=(16, 8))

plt.subplot(1, 2, 1)
plt.imshow(wordcloud_subject, interpolation="bilinear")
plt.axis("off")
plt.title('Word Cloud for Subject Entities')

plt.subplot(1, 2, 2)
plt.imshow(wordcloud_object, interpolation="bilinear")
plt.axis("off")
plt.title('Word Cloud for Object Entities')

plt.show()

###
###
###


TypeError: string indices must be integers

In [ ]:
data = pd.read_csv('dataset/train/train.csv')


In [ ]:
import pickle as pickle
import os
import pandas as pd
import torch
import sklearn
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer
# from load_data import *
import numpy as np
import random

In [22]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [31]:
def tokenized_dataset(dataset, tokenizer):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)
  tokenized_sentences = tokenizer(
      concat_entity,
      list(dataset['sentence']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=256,
      add_special_tokens=True,
      )
  return tokenized_sentences


In [37]:
tokenized_test = []
for i in range(len(data['sentence'])):
    tokenized_test.append(tokenizer.tokenize(data['sentence'][i]))




In [38]:
df_tokenized_test = pd.DataFrame(tokenized_test)
df_tokenized_test.to_csv('tokenized_test.csv', index=True)


In [34]:
print(tokenzied_testsents)

['〈', 'So', '##me', '##th', '##ing', '〉', '는', '조지', '해리', '##슨', '##이', '쓰', '##고', '비틀즈', '##가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']


In [43]:
random_numbers = random.sample(range(len(tokenized_test)), 50)

for i in random_numbers:
    print(data['sentence'][i])
    print(tokenized_test[i])
    print("\n")


1942년 경성치과전문학교 입학을 하였고 1943년 2월에서 1945년 2월까지 2년간 휴학 끝에 1945년 2월 경성치과전문학교 예과 2년에 복학 후 이듬해 1946년 경성치전(경성치과전문학교)이 서울대학교 치의과대학으로 개편되면서 서울대학교 치의과대학 본과에 편입을 하였는데 바로 그 해, 서울대학교 치과대학 본과 1년 재학 시절인 1946년 교내 연극반 반원으로 연극에 입문하였고 이후 한편으로 서울대학교 치과대학 본과 3년 시절이던 1948년 11월에 육군 군의무관 중위 임관하였고 1950년 2월 서울대학교 치의과대학 학사 학위하고 한 달 지난 1950년 3월에 육군 군의무관 대위 진급하였으며 이후 1950년 6월 25일에서 1951년 11월에 전역을 할 때까지 한국 전쟁에 육군 군의무관으로 참전했으며, 1951년 11월에 육군 군의무관 대위 예편하였다.
['1942', '##년', '경성', '##치', '##과', '##전', '##문학', '##교', '입학', '##을', '하', '##였', '##고', '1943', '##년', '2', '##월', '##에서', '1945', '##년', '2', '##월', '##까', '##지', '2', '##년', '##간', '휴학', '끝', '##에', '1945', '##년', '2', '##월', '경성', '##치', '##과', '##전', '##문학', '##교', '예', '##과', '2', '##년', '##에', '복', '##학', '후', '이듬해', '1946', '##년', '경성', '##치', '##전', '(', '경성', '##치', '##과', '##전', '##문학', '##교', ')', '이', '서울대', '##학교', '치', '##의', '##과', '##대', '##학', '##으로', '개편', '##되', '##면서', '서울대', '##학교', '치', '##의', '##과', '##대', '##학', '본', '##과', '##에', '편입', '

In [48]:
# for i in range(len(tokenized_test)):
#     print(data['sentence'][i])
#     print(tokenized_test[i])
#     print("\n")

with open('tokencompare.txt', 'w') as file:
    for i in range(len(tokenized_test)):
        print(data['sentence'][i], file=file)
        print(tokenized_test[i], file=file)
        print(f"{data['subject_entity'][i]},{data['object_entity'][i]}", file=file)
        print("\n", file=file)


In [49]:
with open('sentences.txt', 'w') as file:
    for sentence in data['sentence']:
        file.write(sentence + '\n')


In [55]:
# object_entity_types = data['object_entity']['type'].unique()
# print(object_entity_types)

print(data['object_entity'].keys())

RangeIndex(start=0, stop=32470, step=1)


In [59]:
datatype = {}
for i in range(len(data['object_entity'])):
    datatype.update(data['object_entity'][i][type])

TypeError: string indices must be integers

In [65]:

object_entity_dict = dict(data['object_entity'][0])

print(object_entity_dict['type'])


ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [66]:
import ast

# Convert string representation of dictionaries to actual dictionaries
data['object_entity'] = data['object_entity'].apply(ast.literal_eval)

# Extract the 'type' key from each dictionary
types = data['object_entity'].apply(lambda x: x['type'])

# Get the unique values of the 'type' key
unique_types = types.unique()

print(unique_types)


['PER' 'ORG' 'DAT' 'LOC' 'POH' 'NOH']


In [67]:
import ast

# Convert string representation of dictionaries to actual dictionaries
data['subject_entity'] = data['subject_entity'].apply(ast.literal_eval)

# Extract the 'type' key from each dictionary
types = data['subject_entity'].apply(lambda x: x['type'])

# Get the unique values of the 'type' key
unique_types = types.unique()

print(unique_types)

['ORG' 'PER']


In [68]:
testdata = pd.read_csv('dataset/test/test_data.csv')


In [69]:
import ast

# Convert string representation of dictionaries to actual dictionaries
testdata['object_entity'] = testdata['object_entity'].apply(ast.literal_eval)

# Extract the 'type' key from each dictionary
types = testdata['object_entity'].apply(lambda x: x['type'])

# Get the unique values of the 'type' key
unique_types = types.unique()

print(unique_types)

['ORG' 'POH' 'DAT' 'PER' 'LOC' 'NOH']


In [70]:
import ast

# Convert string representation of dictionaries to actual dictionaries
testdata['subject_entity'] = testdata['subject_entity'].apply(ast.literal_eval)

# Extract the 'type' key from each dictionary
types = testdata['subject_entity'].apply(lambda x: x['type'])

# Get the unique values of the 'type' key
unique_types = types.unique()

print(unique_types)

['ORG' 'PER' 'LOC']
